In [ ]:
#Breakdown of the simple CNN

# Convolutional Layers 

# These layers are the core of the CNN, designed to learn hierarchical feature representations of the input images. 
#The first convolutional layer starts with RGB images and increases the depth (32, 64, 128). 
#This increase in depth allows the network to learn more complex features at each level, 
#starting from basic edges and textures to more complex patterns.
# Each convolutional layer uses a kernel size of 3 with padding set to 1, which helps maintain the 
#spatial dimensions of the output while allowing the network to learn spatial hierarchies in the data.
# The stride is kept at 1 to process the image in fine steps, capturing detailed features without losing much information between layers.

# Pooling Layer

# Max pooling is used to reduce the spatial dimensions of the output from the convolutional layers. 
#By using a 2x2 window with a stride of 2, it reduces the height and width of the feature maps by half, 
#decreasing the amount of computation needed in deeper layers and controlling overfitting

# Fully Connected Layers

# After the convolutional and pooling layers have extracted and condensed the features from the input image, 
#the fully connected layers are used to perform classification based on those features.
# The first fully connected layer reduces the dimensionality from the flattened convolutional layer
#outputs to a smaller feature space, before connecting to the final layer which outputs the predictions across the # of classes

# RELU

# ReLU used after each convolutional and fully connected layer 
#except the last one to introduce non-linearity into the model, allowing it to learn complex patterns. 
#ReLU is used to mitigate the vanishing gradient problem compared to other activation functions like sigmoid or tanh.

# Dropout

# Dropout is used to prevent overfitting by randomly setting a fraction of input units to 0 at each update during training time, 
#which helps by preventing it from relying too much on any one node.
# The dropout layer is specifically placed after the first fully connected layer to regularize the network

In [28]:
import pandas as pd
import os
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Cuda")
else:
    print("CPU")


# Number of epochs
num_epochs = 10

# For storing the loss and accuracy to plot later
train_losses = []
train_accuracies = []



train_csv_path = 'train.csv'
test_csv_path = 'test.csv'

# Load the data
train_df = pd.read_csv(train_csv_path, delimiter='\t', skipinitialspace=True)
test_df = pd.read_csv(test_csv_path, delimiter='\t', skipinitialspace=True)

# Define the dataset class
class FashionDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform
        self.label_mapping = {label: idx for idx, label in enumerate(dataframe.iloc[:, 1].unique())}

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Assuming image ids are in the first column
        img_id = self.dataframe.iloc[idx, 0]
        img_name = os.path.join(self.image_dir, f"{img_id}.jpg")
        image = Image.open(img_name).convert('RGB')  # Ensure the image is loaded in RGB mode
        label_name = self.dataframe.iloc[idx, 1]
        label = self.label_mapping[label_name]
        
        if self.transform:
            image = self.transform(image)
        
        # Ensure the image has 3 channels
        if image.shape[0] != 3:
            raise ValueError(f"Image at index {idx} does not have 3 channels after transformation.")
    
        return image, label

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    # transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
     
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
# Create the dataset
train_dataset = FashionDataset(dataframe=train_df, image_dir='archive/images', transform=transform)
test_dataset = FashionDataset(dataframe=test_df, image_dir='archive/images', transform=transform)

# Create the dataloaders
batch_size = 32
validation_ratio = 0.1
num_train_examples = len(train_dataset)
num_validation_examples = int(num_train_examples * validation_ratio)
num_train_examples -= num_validation_examples
train_subset, validation_subset = random_split(train_dataset, [num_train_examples, num_validation_examples])


train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
validation_loader = DataLoader(validation_subset, batch_size=batch_size, shuffle=False)

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)  # Adjust dropout rate as needed


    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 128 * 16 * 16)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Define the model
num_classes = 13  
model = SimpleCNN(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function for the training step
def train(model, criterion, optimizer, dataloader, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, targets) in enumerate(dataloader):
        # Move tensors to the configured device
        data = data.to(device)
        targets = targets.to(device)

        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = 100 * correct / total

    print(f'Train loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)

def validate(model, criterion, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():  # No need to track the gradients
        for batch_idx, (data, targets) in enumerate(dataloader):
            data = data.to(device)
            targets = targets.to(device)

            outputs = model(data)
            loss = criterion(outputs, targets)

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = 100 * correct / total

    print(f'Validation loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')


# Function to evaluate the model on the test set
def test(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for data, targets in dataloader:
            data = data.to(device)
            targets = targets.to(device)

            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

Cuda


In [29]:
# Function for the validation step

# Add validation to the training epochs
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    train(model, criterion, optimizer, train_loader, device)
    validate(model, criterion, validation_loader, device)

# Save the model checkpoint
torch.save(model.state_dict(), 'fashion_model.pth')


Epoch 1/10
Train loss: 0.4434, Accuracy: 86.71%
Validation loss: 0.2252, Accuracy: 92.93%
Epoch 2/10
Train loss: 0.2163, Accuracy: 93.59%
Validation loss: 0.1839, Accuracy: 94.36%
Epoch 3/10
Train loss: 0.1556, Accuracy: 95.29%
Validation loss: 0.1669, Accuracy: 94.96%
Epoch 4/10
Train loss: 0.1218, Accuracy: 96.32%
Validation loss: 0.1788, Accuracy: 95.10%
Epoch 5/10
Train loss: 0.0902, Accuracy: 97.12%
Validation loss: 0.2118, Accuracy: 94.93%
Epoch 6/10
Train loss: 0.0749, Accuracy: 97.58%
Validation loss: 0.1960, Accuracy: 94.91%
Epoch 7/10
Train loss: 0.0628, Accuracy: 97.96%
Validation loss: 0.1902, Accuracy: 95.82%
Epoch 8/10
Train loss: 0.0499, Accuracy: 98.43%
Validation loss: 0.2141, Accuracy: 95.15%
Epoch 9/10
Train loss: 0.0483, Accuracy: 98.45%
Validation loss: 0.2453, Accuracy: 95.75%
Epoch 10/10
Train loss: 0.0423, Accuracy: 98.65%
Validation loss: 0.2618, Accuracy: 95.80%


In [30]:
# Load the model (for evaluation)
model.load_state_dict(torch.load('fashion_model.pth'))

# Evaluate the model
test(model, test_loader, device)

Test Accuracy: 28.93%


In [ ]:
# Baseline Model Performance Conclusion

# The baseline approach's performance on the training and validation datasets shows a consistent improvement over the epochs, which is a a great indication that the model is learning

# Training Performance

#The model's accuracy on the training set shows an upward trend from 86.71% to 98.65% over 10 epochs. 
#Similarly, the training loss decreased from 0.4434 to 0.0423. 
#This suggests that the model is fitting well to the training data and learning the underlying patterns effectively.

# Validation Performance

#The validation accuracy started at a high of 92.93% and increased to 95.80% by the 10th epoch, 
#while the validation loss initially decreased but then started to rise from the 4th epoch onward, reaching 0.2618. 
#The increasing loss alongside increasing accuracy may indicate that while the model is getting better at classifying correctly, 
#it is also becoming more confident in its incorrect predictions, which could be a sign of overfitting.

# Test Performance

#significantly lower test accuracy of 28.93%. This discrepancy is indicative of a model that has not generalized well to unseen data.

# Results

# Overfitting: The improving training accuracy and decreasing loss are usually good signs
#the increasing validation loss suggests the model might be overfitting. 
#Overfitting occurs when a model learns the details and noise in the training data to an extent that it 
#negatively impacts the model's performance on new data.

#The significant difference between validation accuracy and test accuracy is concerning. 
#This could be due to a variety of factors such as

# A difference in the distribution of data between training/validation and testing datasets - unlikly given how the assignment was set up
# The model might have memorized specific features of the training set that aren't as prevalent or are represented differently in the test set.
# There might be a data leakage in the training/validation set, or 
#the test set could be too challenging or not well-represented by the model's learned features.

#Given the high performance on the training set and the lower performance on the test set, 
#it's clear that the model could benefit from better regularization techniques. 
#Regularization can help to reduce overfitting by penalizing overly complex models and 
#promoting simpler hypotheses that may generalize better to unseen data.

# Data Augmentation or Additional Data: To help the model generalize better, - which will be done in later part of the assignment

# In conclusion, while the model shows promise in learning from the training data, 
#the poor test accuracy highlights a need for strategies to improve generalization, 
 
#and ensuring that the model has not overfit to the training and validation data.

In [ ]:
# Enhanced Mode - Adjusting Dropout Rate
# Why Tune? 
#The dropout rate determines how much information is "forgotten" during a training iteration. 
#Finding the right balance is key to achieving good generalization without losing necessary information.

# I think the model is overfitting so i will try increasing the dropout rate.

#Hopefully this will help control my overfitting issue

In [32]:
import pandas as pd
import os
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Cuda")
else:
    print("CPU")


# Number of epochs
num_epochs = 10

# For storing the loss and accuracy to plot later
train_losses = []
train_accuracies = []



train_csv_path = 'train.csv'
test_csv_path = 'test.csv'

# Load the data
train_df = pd.read_csv(train_csv_path, delimiter='\t', skipinitialspace=True)
test_df = pd.read_csv(test_csv_path, delimiter='\t', skipinitialspace=True)

# Define the dataset class
class FashionDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform
        self.label_mapping = {label: idx for idx, label in enumerate(dataframe.iloc[:, 1].unique())}

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Assuming image ids are in the first column
        img_id = self.dataframe.iloc[idx, 0]
        img_name = os.path.join(self.image_dir, f"{img_id}.jpg")
        image = Image.open(img_name).convert('RGB')  # Ensure the image is loaded in RGB mode
        label_name = self.dataframe.iloc[idx, 1]
        label = self.label_mapping[label_name]
        
        if self.transform:
            image = self.transform(image)
        
        # Ensure the image has 3 channels
        if image.shape[0] != 3:
            raise ValueError(f"Image at index {idx} does not have 3 channels after transformation.")
    
        return image, label

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    # transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
     
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
# Create the dataset
train_dataset = FashionDataset(dataframe=train_df, image_dir='archive/images', transform=transform)
test_dataset = FashionDataset(dataframe=test_df, image_dir='archive/images', transform=transform)

# Create the dataloaders
batch_size = 32
validation_ratio = 0.1
num_train_examples = len(train_dataset)
num_validation_examples = int(num_train_examples * validation_ratio)
num_train_examples -= num_validation_examples
train_subset, validation_subset = random_split(train_dataset, [num_train_examples, num_validation_examples])


train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
validation_loader = DataLoader(validation_subset, batch_size=batch_size, shuffle=False)

class DropoutModel(nn.Module):
    def __init__(self, num_classes):
        super(DropoutModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.7)  # Adjust dropout rate as needed


    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 128 * 16 * 16)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Define the model
num_classes = 13  
model = DropoutModel(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function for the training step
def train(model, criterion, optimizer, dataloader, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, targets) in enumerate(dataloader):
        # Move tensors to the configured device
        data = data.to(device)
        targets = targets.to(device)

        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = 100 * correct / total

    print(f'Train loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)

def validate(model, criterion, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():  # No need to track the gradients
        for batch_idx, (data, targets) in enumerate(dataloader):
            data = data.to(device)
            targets = targets.to(device)

            outputs = model(data)
            loss = criterion(outputs, targets)

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = 100 * correct / total

    print(f'Validation loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')


# Function to evaluate the model on the test set
def test(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for data, targets in dataloader:
            data = data.to(device)
            targets = targets.to(device)

            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

Cuda


In [33]:
# Function for the validation step

# Add validation to the training epochs
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    train(model, criterion, optimizer, train_loader, device)
    validate(model, criterion, validation_loader, device)

# Save the model checkpoint
torch.save(model.state_dict(), 'fashion_dropout.pth')


Epoch 1/10
Train loss: 0.4896, Accuracy: 85.18%
Validation loss: 0.2819, Accuracy: 92.31%
Epoch 2/10
Train loss: 0.2638, Accuracy: 92.12%
Validation loss: 0.2205, Accuracy: 93.62%
Epoch 3/10
Train loss: 0.2058, Accuracy: 94.02%
Validation loss: 0.1909, Accuracy: 94.44%
Epoch 4/10
Train loss: 0.1638, Accuracy: 95.10%
Validation loss: 0.1988, Accuracy: 94.56%
Epoch 5/10
Train loss: 0.1337, Accuracy: 95.90%
Validation loss: 0.2190, Accuracy: 94.31%
Epoch 6/10
Train loss: 0.1199, Accuracy: 96.30%
Validation loss: 0.1824, Accuracy: 95.20%
Epoch 7/10
Train loss: 0.1029, Accuracy: 96.80%
Validation loss: 0.1821, Accuracy: 95.47%
Epoch 8/10
Train loss: 0.0864, Accuracy: 97.36%
Validation loss: 0.2131, Accuracy: 95.08%
Epoch 9/10
Train loss: 0.0794, Accuracy: 97.51%
Validation loss: 0.2118, Accuracy: 95.72%
Epoch 10/10
Train loss: 0.0727, Accuracy: 97.80%
Validation loss: 0.2068, Accuracy: 95.18%


In [34]:
# Load the model (for evaluation)
model.load_state_dict(torch.load('fashion_dropout.pth'))

# Evaluate the model
test(model, test_loader, device)

Test Accuracy: 28.57%


In [38]:
# Dropout Model Performance Conclusion

#aimed to address overfitting observed in the baseline model by regularizing the network during training. 

# Training and Validation Performance
# Decrease in Training Loss: The model shows a consistent decrease in training loss from 0.4896 to 0.0727 over 10 epochs, 
#indicating that the model is learning effectively from the training data.

# Increase in Training Accuracy:
#Similarly, training accuracy improves from 85.18% to 97.80%, 
#demonstrating the model's increasing proficiency in classifying the training data correctly.

# Validation Loss and Accuracy Fluctuations: 
#The validation loss decreases initially but shows fluctuations in later epochs, 
#with an increase in epochs 5, 8, and 9 compared to earlier epochs. Despite these fluctuations, 
#the validation accuracy generally remains high, peaking at 95.72% in epoch 9, 
#suggesting that the model retains a good level of generalization on unseen data.

# Analysis of Dropout Impact
# The implemented dropout appears to have a positive impact on the model's ability to generalize, as indicated by high validation accuracy. 
#However, the fluctuating validation loss suggests that the model's confidence in its 
#predictions varies significantly across epochs. This could be an indicator of the model still 
#overfitting to some extent or being affected by the increased regularization in a way that impacts its loss metrics more than its accuracy.

# Test Performance
# Significant Discrepancy: There is a stark discrepancy between the validation accuracy and test accuracy, 
#with the latter being significantly lower at 28.57%. This huge gap indicates a problem with model generalization to the test set.

# Conclusions and Recommendations
#Increasing the dropout hurt the models performence but the use of dropout helped the model

In [35]:
import pandas as pd
import os
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Cuda")
else:
    print("CPU")


# Number of epochs
num_epochs = 10

# For storing the loss and accuracy to plot later
train_losses = []
train_accuracies = []



train_csv_path = 'train.csv'
test_csv_path = 'test.csv'

# Load the data
train_df = pd.read_csv(train_csv_path, delimiter='\t', skipinitialspace=True)
test_df = pd.read_csv(test_csv_path, delimiter='\t', skipinitialspace=True)

# Define the dataset class
class FashionDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform
        self.label_mapping = {label: idx for idx, label in enumerate(dataframe.iloc[:, 1].unique())}

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Assuming image ids are in the first column
        img_id = self.dataframe.iloc[idx, 0]
        img_name = os.path.join(self.image_dir, f"{img_id}.jpg")
        image = Image.open(img_name).convert('RGB')  # Ensure the image is loaded in RGB mode
        label_name = self.dataframe.iloc[idx, 1]
        label = self.label_mapping[label_name]
        
        if self.transform:
            image = self.transform(image)
        
        # Ensure the image has 3 channels
        if image.shape[0] != 3:
            raise ValueError(f"Image at index {idx} does not have 3 channels after transformation.")
    
        return image, label

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    # transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
     
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.RandomRotation(10),  # Randomly rotate images by up to 10 degrees
    transforms.RandomResizedCrop(128, scale=(0.8, 1.0)),  # Randomly crop and resize images
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Randomly change brightness, contrast, and saturation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
# Create the dataset
train_dataset = FashionDataset(dataframe=train_df, image_dir='archive/images', transform=train_transforms)
test_dataset = FashionDataset(dataframe=test_df, image_dir='archive/images', transform=transform)

# Create the dataloaders
batch_size = 32
validation_ratio = 0.1
num_train_examples = len(train_dataset)
num_validation_examples = int(num_train_examples * validation_ratio)
num_train_examples -= num_validation_examples
train_subset, validation_subset = random_split(train_dataset, [num_train_examples, num_validation_examples])


train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
validation_loader = DataLoader(validation_subset, batch_size=batch_size, shuffle=False)

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)  


    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 128 * 16 * 16)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Define the model
num_classes = 13  
model = SimpleCNN(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function for the training step
def train(model, criterion, optimizer, dataloader, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, targets) in enumerate(dataloader):
        # Move tensors to the configured device
        data = data.to(device)
        targets = targets.to(device)

        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = 100 * correct / total

    print(f'Train loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)

def validate(model, criterion, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():  # No need to track the gradients
        for batch_idx, (data, targets) in enumerate(dataloader):
            data = data.to(device)
            targets = targets.to(device)

            outputs = model(data)
            loss = criterion(outputs, targets)

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = 100 * correct / total

    print(f'Validation loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')


# Function to evaluate the model on the test set
def test(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for data, targets in dataloader:
            data = data.to(device)
            targets = targets.to(device)

            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

Cuda


In [36]:
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    train(model, criterion, optimizer, train_loader, device)
    validate(model, criterion, validation_loader, device)

torch.save(model.state_dict(), 'fashion_Aug.pth')

Epoch 1/10
Train loss: 0.7395, Accuracy: 76.48%
Validation loss: 0.4363, Accuracy: 86.92%
Epoch 2/10
Train loss: 0.4468, Accuracy: 86.02%
Validation loss: 0.3411, Accuracy: 89.59%
Epoch 3/10
Train loss: 0.3706, Accuracy: 88.20%
Validation loss: 0.2945, Accuracy: 90.85%
Epoch 4/10
Train loss: 0.3389, Accuracy: 89.45%
Validation loss: 0.2808, Accuracy: 90.95%
Epoch 5/10
Train loss: 0.3168, Accuracy: 90.21%
Validation loss: 0.2829, Accuracy: 91.27%
Epoch 6/10
Train loss: 0.2995, Accuracy: 90.83%
Validation loss: 0.2689, Accuracy: 91.67%
Epoch 7/10
Train loss: 0.2896, Accuracy: 91.05%
Validation loss: 0.2618, Accuracy: 92.11%
Epoch 8/10
Train loss: 0.2660, Accuracy: 91.66%
Validation loss: 0.2418, Accuracy: 93.05%
Epoch 9/10
Train loss: 0.2592, Accuracy: 91.82%
Validation loss: 0.2212, Accuracy: 92.90%
Epoch 10/10
Train loss: 0.2579, Accuracy: 91.81%
Validation loss: 0.2250, Accuracy: 93.03%


In [37]:

model.load_state_dict(torch.load('fashion_Aug.pth'))

test(model, test_loader, device)

Test Accuracy: 29.00%


In [ ]:

# Conclusion on the Performance - Data Augmentation

# The experiment results from incorporating data augmentation into the training process demonstrate both its strengths and limitations 
#in enhancing model performance. Data augmentation was intended to increase the model's ability to generalize by introducing a wider 
#variety of training examples without the need for additional data collection. 

# Training and Validation Performance
# Initial Increase in Training Loss: 

#The first epoch starts with a higher training loss (0.7395) compared to models without data augmentation. 
#This is expected, as data augmentation introduces more variability and complexity into the training process, 
#making the initial learning phase more challenging.

# Steady Improvement: 

#Both training loss and accuracy show consistent improvement over the epochs, with training loss decreasing from 0.7395 to 0.2579, 
#and accuracy increasing from 76.48% to 91.81%. This indicates that the model effectively learns from the augmented data over time.

# Validation Performance: 

#The validation loss decreases from 0.4363 to 0.2250 across epochs, and validation accuracy improves, reaching a peak of 93.05% in epoch 8 before 
#stabilizing around 93%. The improvement in validation metrics suggests that data augmentation has helped enhance the model's generalization 
#capabilities.

# Test Performance
# Marginal Improvement: 

#The test accuracy stands at 29.00%, which, while slightly better than the dropout model's performance (28.57%), still indicates a significant 
#generalization gap. This discrepancy highlights that while data augmentation and dropout regularization can mitigate overfitting to some extent,
#additional strategies may be required to bridge the gap between training/validation performance and test set performance.

# Analysis and Recommendations
# Data Augmentation's Impact: The consistent improvement in training and validation metrics confirms the beneficial impact of data 
#augmentation on model learning and generalization. However, the marginal increase in test accuracy suggests limits to its effectiveness alone.

# Generalization Gap: 

#The stark difference between validation and test performance suggests potential issues beyond model overfitting. 
#It could point towards a distribution shift between training/validation data and test data, or 
#it may indicate that the model's capacity is not adequately aligned with the complexity of the test data.

# Further Experimentation: 
#Experimenting with different or additional data augmentation techniques could provide further insights into their impact on model
#performance. Moreover, exploring other regularization techniques, adjusting model architecture, or using advanced training strategies 
#like transfer learning could offer additional pathways to improve test accuracy.

# Review Dataset Split: 
#It's crucial to ensure the dataset is split in a manner that accurately reflects the distribution of data the 
#model will encounter in practice. A review of the splitting process may reveal opportunities for improvement.

# In conclusion, while data augmentation has positively impacted training dynamics and validation performance, 
#bridging the significant gap to test performance requires a multifaceted approach. 
#This might include further experimentation with data processing, model architecture adjustments, and exploring additional or more sophisticated 
#regularization and augmentation strategies.